In [2]:
import glob
import time
import pandas as pd
# from xml.dom import minidom
from lxml import etree
from nltk import ngrams
from nltk.tokenize import sent_tokenize
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.stem import PorterStemmer

[nltk_data] Downloading package punkt to /home/ebryx/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/ebryx/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/ebryx/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [3]:
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 

In [4]:
from sklearn.preprocessing import MultiLabelBinarizer

In [5]:
from sklearn.model_selection import train_test_split

In [6]:
t1=time.time()
path='./dataset/blogs_train/'
all_files = glob.glob(path + "/*.xml")
li = []

for filename in all_files:
  # print("file name ", filename)
  li.append(filename)
    

t2=time.time()
print("time taken to read All csv :: ", t2-t1, " seconds ")

time taken to read All csv ::  0.0993509292602539  seconds 


In [7]:
def clean_file_names(li):
  # names_list=[]
  # for i in range(len(li)):
  name_list=li.split('/')[-1]
  return name_list

In [8]:


def get_attributes_from_name(name):
  # data_object=[]
  # for data in range(len(names_list)):
  #   dummy_var=names_list[data].split('.')
  #   dummy_object={'id':dummy_var[0], 'gender':dummy_var[1],
  #                 'age':dummy_var[2], 'category':dummy_var[3],
  #                 'horoscope':dummy_var[4]
  #                 }
  #   data_object.append(dummy_object)
  dummy_var=name.split('.')
  data_object={'id':dummy_var[0], 'gender':dummy_var[1],
                  'age':dummy_var[2], 'category':dummy_var[3],
                  'horoscope':dummy_var[4]
                }
  
  return data_object



In [9]:
parser = etree.XMLParser(encoding='ISO-8859-1', recover = True, huge_tree=True)

In [10]:
def read_xml(li):
    
    
    all_objects=[]
    for i in range(len(li)):
    # print("i will run ", i+1," times")
        name_of_file=clean_file_names(li[i])
    # print(name_of_file)
        data_object=get_attributes_from_name(name_of_file)
    # print(data_object)

        post_text='  '
        tree = etree.parse(li[i], parser=parser)
        root = tree.getroot()
        for element in root.iter():
            if element.tag == 'post':
                element_text = element.text
                post_text = post_text + element_text
        mystring = post_text.replace('\n', '').replace('\r', '').replace('\t', '')
        mystring = mystring.replace('   ', '').replace('\\','')
   


   
    # print(mystring)
    # name_of_file=clean_file_names(li[i])
    # data_object=get_attributes_from_name(name_of_file)
    



        data_object['posts']=mystring   
        all_objects.append(data_object)

    return all_objects

In [11]:
t1=time.time()
data=read_xml(li[0:100])
t2=time.time()

In [12]:
stop_words = set(stopwords.words('english')) 
def remove_stop_words(posts):
    filtered_post=[]
    
    word_tokens = word_tokenize(posts) 
    filtered_sentence = []   
    for w in word_tokens: 
        if w not in stop_words: 
            filtered_sentence.append(w)

    filtered_post.append(filtered_sentence)
    return filtered_post

In [13]:
stop_words = set(stopwords.words('english')) 
def fun_remove_stop_words(posts):
    filtered=''
    for x in posts.split(' '):
        if x not in stop_words:
            filtered+=' '+x
    return filtered    

In [14]:
for i in range(len(data)):
    data[i]['cleaned_stop_words']=fun_remove_stop_words(data[i]['posts'])

In [15]:
def remove_dashes(data):
    for i in range(len(data)):
        data[i]['cleaned_stop_words']=data[i]['cleaned_stop_words'].replace('--', ' ')
        

In [16]:
remove_dashes(data)

In [53]:
df=pd.DataFrame(data)

In [21]:
X=df[['age','cleaned_stop_words']]

In [22]:
dummy_variable_1 = pd.get_dummies(df["category"])
X=pd.concat([X,dummy_variable_1], axis=1)

In [23]:
del dummy_variable_1

In [24]:
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np

In [25]:
vectorizer = CountVectorizer()
vectorizer.fit(X['cleaned_stop_words'])
posts = vectorizer.transform(X['cleaned_stop_words'])


In [26]:
posts

<100x35680 sparse matrix of type '<class 'numpy.int64'>'
	with 137673 stored elements in Compressed Sparse Row format>

In [27]:
transformed_posts = np.array([row.toarray() for row in posts])

In [32]:
transformed_posts.shape=(100,35680)

In [33]:
del df,posts

NameError: name 'df' is not defined

In [34]:
transformed_posts=pd.DataFrame(transformed_posts)

In [35]:
X=pd.concat([X,transformed_posts],axis=1)

In [38]:
del transformed_posts

In [42]:
X=X.drop(['cleaned_stop_words'], axis=1)

In [43]:
X

,age,Advertising,Arts,Banking,BusinessServices,Communications-Media,Education,Engineering,Government,HumanResources,...,35670,35671,35672,35673,35674,35675,35676,35677,35678,35679
0,15,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,25,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,17,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,24,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,17,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,25,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
96,23,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
97,16,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
98,24,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [48]:
X['age']=X['age'].astype(int)

In [50]:
from scipy import stats

In [51]:
t1=time.time()

X['age'] = stats.zscore(X['age'])
    
t2=time.time()
print("time ::: ", t2-t1 , " seconds to find z score")

time :::  0.001110076904296875  seconds to find z score


In [52]:
X.head()

,age,Advertising,Arts,Banking,BusinessServices,Communications-Media,Education,Engineering,Government,HumanResources,...,35670,35671,35672,35673,35674,35675,35676,35677,35678,35679
0,-1.006980,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0.216568,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,-0.762271,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0.094213,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,-0.762271,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


,age,category,posts
0,15,Student,"i know, i need to be getting sleep; i know, i..."
1,25,indUnk,love studying?can't get enough of those fat b...
2,17,Education,Im currently updating my site. And almost doin...
3,24,Student,So I was a witness to a pretty nasty accident ...
4,17,indUnk,ok so we skipped multimedia computer lecture...


In [ ]:
del X, X2

In [ ]:
df.head()

In [ ]:
X=df[['cleaned_stop_words']]

In [ ]:
X

In [49]:
X.dtypes

age                 int64
Advertising         uint8
Arts                uint8
Banking             uint8
BusinessServices    uint8
                    ...  
35675               int64
35676               int64
35677               int64
35678               int64
35679               int64
Length: 35700, dtype: object

In [ ]:
dummy_variable_1 = pd.get_dummies(df["category"])
df=pd.concat([df,dummy_variable_1], axis=1)

In [ ]:
df.columns

In [ ]:
df_2=pd.concat([df['age'],df['posts']], axis=1)

In [ ]:
i=0
for x in df.columns:
    i+=1
    if i>7:
        df_2=pd.concat([df_2,df[x]], axis=1)

In [ ]:
df_2.tail()

In [ ]:
columns=df_2.columns[2:]

In [ ]:
df_2.head()

In [ ]:
X=df_2['posts']

In [ ]:
X2.shape

In [ ]:
pd.concat([X, X2], join="inner")

In [ ]:
X.shape

In [ ]:
X2=pd.concat([df_2['posts'],df_2[columns]], axis=1)

In [ ]:
X2.dtypes

In [19]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
vectorizer = CountVectorizer()
# vectorizer.fit(X['posts'])
# X['posts'] = vectorizer.transform(X['posts'])
transformed_posts = np.array([row.toarray() for row in vectorizer.fit_transform(X['posts'])])

In [33]:
vectorizer = CountVectorizer()
vectorizer.fit(X['category'])
X['category'] = vectorizer.transform(X['category'])


/home/ebryx/anaconda3/envs/tf/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [21]:
import numpy as np

In [22]:
vectorizer = CountVectorizer()
# vectorizer.fit(X['category'])
# X['category'] = vectorizer.transform(X['category'])
transformed_data = np.array([row.toarray() for row in vectorizer.fit_transform(X['category'])])

In [23]:
transformed_data.shape=(14600,45)

In [24]:
transformed_data=pd.DataFrame(transformed_data)

In [25]:
X2=pd.concat([X,transformed_data],axis=1)

In [26]:
X2

,age,category,posts,0,1,2,3,4,5,6,...,35,36,37,38,39,40,41,42,43,44
0,15,Student,"i know, i need to be getting sleep; i know, i...",0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1,25,indUnk,love studying?can't get enough of those fat b...,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,17,Education,Im currently updating my site. And almost doin...,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,24,Student,So I was a witness to a pretty nasty accident ...,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
4,17,indUnk,ok so we skipped multimedia computer lecture...,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14595,15,Student,ok we got webspace off abp for bgs but of cour...,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
14596,15,indUnk,"dearest diary, i do not know wad to put for t...",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
14597,16,indUnk,Good friends are the greatest treasures of th...,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
14598,23,Student,WE BRING THIS SPECIAL ANNOUCEMENT TO YOU LIVE...,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [ ]:
df_2.columns

In [ ]:
df_2.head()

In [54]:
labels= []
for hor,gen in zip(df['horoscope'],df['gender']):
    labels.append((hor,gen))

In [55]:
mlb = MultiLabelBinarizer()
mlb.fit(labels)
classes=len(mlb.classes_)
print("classes  :: ", classes)
mlb.classes_

classes  ::  14


array(['Aquarius', 'Aries', 'Cancer', 'Capricorn', 'Gemini', 'Leo',
       'Libra', 'Pisces', 'Sagittarius', 'Scorpio', 'Taurus', 'Virgo',
       'female', 'male'], dtype=object)

In [ ]:
trainX.shape

In [ ]:
X=X.transpose()

In [56]:
labels = mlb.fit_transform(labels)

In [57]:
y=labels

In [58]:
trainX, testX, trainY, testY = train_test_split(X,y, test_size=0.2, random_state=42)

In [59]:
from keras.models import Sequential
from keras import layers
from keras.preprocessing.text import Tokenizer
from keras.wrappers.scikit_learn import KerasClassifier

Using TensorFlow backend.


In [60]:


input_dim = trainX.shape[1]  # Number of features

model = Sequential()
model.add(layers.Dense(20, input_dim=input_dim, activation='relu'))
model.add(layers.Dense(10,  activation='tanh'))
# model.add(layers.Flatten())
model.add(layers.Dense(1024, activation='relu'))

model.add(layers.BatchNormalization())
model.add(layers.Dropout(0.5))
model.add(layers.Dense(classes, activation='softmax'))
# model.add(Activation('softmax'))







Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [62]:
model.compile(loss='binary_crossentropy', 
              optimizer='adam', 
              metrics=['accuracy'])
model.summary()




Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 20)                714020    
_________________________________________________________________
dense_2 (Dense)              (None, 10)                210       
_________________________________________________________________
dense_3 (Dense)              (None, 1024)              11264     
_________________________________________________________________
batch_normalization_1 (Batch (None, 1024)              4096      
_________________________________________________________________
dropout_1 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_4 (Dense)              (None, 14)                14350     
Total params: 743,940
Trainable params: 741,89

In [63]:
trainX, testX, trainY, testY
classifier_nn = model.fit(trainX,trainY,
                    epochs=5,
                    verbose=True,
                    validation_data=(testX, testY),
                    batch_size=10)






Train on 80 samples, validate on 20 samples
Epoch 1/5





80/80 [==============================] - 3s 33ms/step - loss: 0.5078 - acc: 0.8562 - val_loss: 0.5301 - val_acc: 0.8393
Epoch 2/5
80/80 [==============================] - 0s 2ms/step - loss: 0.4573 - acc: 0.8545 - val_loss: 0.4888 - val_acc: 0.8571
Epoch 3/5
80/80 [==============================] - 0s 2ms/step - loss: 0.4447 - acc: 0.8625 - val_loss: 0.5128 - val_acc: 0.8536
Epoch 4/5
80/80 [==============================] - 0s 2ms/step - loss: 0.3786 - acc: 0.8643 - val_loss: 0.4392 - val_acc: 0.8536
Epoch 5/5
80/80 [==============================] - 0s 2ms/step - loss: 0.3753 - acc: 0.8688 - val_loss: 0.4397 - val_acc: 0.8679


In [73]:
model.predict(testX[1])

ValueError: Error when checking input: expected dense_1_input to have shape (35700,) but got array with shape (1,)

In [78]:
testX[0]

83    0
53    0
70    0
45    1
44    0
39    0
22    0
80    0
10    0
0     0
18    1
30    0
73    1
33    0
90    0
4     0
76    0
77    0
12    0
31    0
Name: 0, dtype: int64

(20,)

In [72]:
trainY

array([[0, 0, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 1, 0],
       ...,
       [1, 0, 0, ..., 0, 0, 1],
       [0, 1, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 0, 0, 1]])

In [ ]:
print("time taken to read :: ", (t2-t1)/100)

In [ ]:
data_backup=data.copy()

### Word2 Vector

In [ ]:
words_of_posts=[]
for j in range(len(data)):
    posts=data[j]['posts']
    
    words=[]
    for i in range(len(posts)):
        words.append(word_tokenize(posts[i]))
    words_of_posts.append(words)

In [ ]:
remove_stop_words(data[1]['posts'])

In [ ]:
if len(data)==len(words_of_posts):
    for i in range(len(data)):
        data[i]['tokenized']=words_of_posts[i]

In [ ]:
df=pd.DataFrame(data)

In [ ]:
X=df['tokenized'].values

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
vectorizer = CountVectorizer()
vectorizer.fit(X)
X = vectorizer.transform(X)

In [ ]:
y=data['horoscope']

In [ ]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(y)

In [ ]:
y_transform=le.transform(y)

In [ ]:
from keras.utils import np_utils

In [ ]:
dummy_y = np_utils.to_categorical(y_transform)

In [ ]:
#Split

In [ ]:
# model

In [ ]:
# compile

In [ ]:
# fit

In [ ]:
def _create_frequency_matrix(sentences):
    frequency_matrix = {}
    stopWords = set(stopwords.words("english"))
    ps = PorterStemmer()

    for sent in sentences:
        freq_table = {}
        words = word_tokenize(sent)
        for word in words:
            word = word.lower()
            word = ps.stem(word)
            if word in stopWords:
                continue

            if word in freq_table:
                freq_table[word] += 1
            else:
                freq_table[word] = 1

        frequency_matrix[sent[:15]] = freq_table

    return frequency_matrix

In [ ]:
freq_matrix=_create_frequency_matrix(data[1]['posts'])

In [ ]:
def _create_tf_matrix(freq_matrix):
    tf_matrix = {}

    for sent, f_table in freq_matrix.items():
        tf_table = {}

        count_words_in_sentence = len(f_table)
        for word, count in f_table.items():
            tf_table[word] = count / count_words_in_sentence

        tf_matrix[sent] = tf_table

    return tf_matrix

In [ ]:
tf_matrix=_create_tf_matrix(freq_matrix)

In [ ]:
def _create_documents_per_words(freq_matrix):
    word_per_doc_table = {}

    for sent, f_table in freq_matrix.items():
        for word, count in f_table.items():
            if word in word_per_doc_table:
                word_per_doc_table[word] += 1
            else:
                word_per_doc_table[word] = 1

    return word_per_doc_table

In [ ]:
doc_per_word=_create_documents_per_words(freq_matrix)

In [ ]:
def _create_idf_matrix(freq_matrix, count_doc_per_words, total_documents):
    idf_matrix = {}

    for sent, f_table in freq_matrix.items():
        idf_table = {}

        for word in f_table.keys():
            idf_table[word] = math.log10(total_documents / float(count_doc_per_words[word]))

        idf_matrix[sent] = idf_table

    return idf_matrix

In [ ]:
_create_idf_matrix(freq_matrix,doc_per_word,total_documents)